# Binary Classification of Insurance Selling

The aim of this workbook is train a model to predict whether customers respond positively to an automobile insurance offer.

url: https://www.kaggle.com/competitions/playground-series-s4e7/overview

# Lib Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
RANDOM_STATE = 32
Y_COLUMN = 'Response'

In [ ]:
import os

TRAIN_DATASET_DIR = '/kaggle/input/playground-series-s4e7/train.csv' if os.path.exists('/kaggle/input/') else 'train.csv'
TEST_DATASET_DIR = '/kaggle/input/playground-series-s4e7/test.csv' if os.path.exists('/kaggle/input/') else 'test.csv'

print(f"train dataset dir: {TRAIN_DATASET_DIR}")
print(f"test dataset dir: {TEST_DATASET_DIR}")

## 1. EDA

### 1.1 Train and Test Dataset Loading

The aim of this section is to load the train and test datasets, in order to investigate missing data and general trends.

In [ ]:
train_df = dd.read_csv(TRAIN_DATASET_DIR)

In [ ]:
train_df = train_df.compute()

In [ ]:
train_df.head()

In [ ]:
test_df = dd.read_csv(TEST_DATASET_DIR)
test_df.head()

Train Data Null and Dtype checks

In [ ]:
train_df.isna().sum()

In [ ]:
train_df[Y_COLUMN].value_counts(normalize=True)

There is a clear bias in this dataset which will need to be address with under / over sampling.

In [ ]:
train_df.dtypes

Test Data Null and Dtype checks

In [ ]:
test_df.isna().sum().compute()

There doesn't seem to be any missing or unusual data so we can proceed with the EDA of the training Dataset.

### 1.2 Train Dataset Feature Engineering

The first step is to analyse the current dataset to determine if features can be engineered. For starters, I'm going to investigate the object columns to determine whether they can be encoded.

In [ ]:
train_df.head()

In [ ]:
train_df['Gender'].value_counts()

There are only two values so these can be binary encoded.

In [ ]:
train_df['Vehicle_Age'].value_counts()

There are only three unique values for Vehicle Age, so these can also be encoded.

In [ ]:
train_df['Vehicle_Damage'].value_counts()

Finally Vehicle Damage can also be binary encoded.

So to summarize, all of the object columns can be encoded. Vehicle Damage and Gender will be Binary Encoded, and Vehicle Age will be one hot encoded.

In [ ]:
BINARY_COLS = ['Gender', 'Vehicle_Damage']
ONE_HOT_COLS = ['Vehicle_Age']

In [ ]:
label_binarizer = LabelBinarizer()

In [ ]:
train_df['Gender'] = train_df['Gender'].apply(lambda x: 0 if x == 'Male' else 1)
train_df['Vehicle_Damage'] = train_df['Vehicle_Damage'].apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
train_df.head()

In [ ]:
vehicle_age_one_hot = pd.get_dummies(train_df['Vehicle_Age'])

In [ ]:
train_df[vehicle_age_one_hot.columns] = vehicle_age_one_hot.values.astype(int)

In [ ]:
train_df.drop(['Vehicle_Age', 'id'], axis=1, inplace=True)

In [ ]:
train_df.head()

### 1.3 Numerical Col Analysis

The next step is to review the distribution of numerical columns

In [ ]:
NUMERICAL_COLS = ['Age', 'Region_Code', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']

In [ ]:
train_df[NUMERICAL_COLS].hist(bins=20, figsize=(12, 10))

Given the above distributions, it may be worth normalizing these columns.

### 1.4 Target vs Feature Analysis

The next step is to review the distribution of the Target Variable, and investigate the relationship between the rest of the features prior to model training.

In [ ]:
plt.matshow(train_df.corr())
plt.show()

## 2. Pipeline Definition and Train Test Split

Define the preprocessors used for numerical, one hot and binary data.

In [ ]:
numerical_transformer = Pipeline(
    steps=[
        ('ss', StandardScaler())
    ]
)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, NUMERICAL_COLS)
    ]
)

In [ ]:
X = train_df.drop(Y_COLUMN, axis=1)
y = train_df[Y_COLUMN]

In [ ]:
ss = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=RANDOM_STATE)
train_index, test_index = next(ss.split(X, y))

In [ ]:
X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

In [ ]:
lr_model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegressionCV(Cs=10, cv=4, penalty='l2', class_weight={0: 0.12, 1: 0.88}, random_state=RANDOM_STATE))
    ]
)

## 3. Initial Model Training and Evaluation

In [ ]:
lr_model.fit(X_train, y_train)

In [ ]:
y_pred = lr_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))